In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import shutil
source_path = "/content/drive/My Drive/test.mp4"
destination_path = "/content/"
shutil.copy(source_path, destination_path)


'/content/test.mp4'

In [ ]:
!yolo track source="/content/drive/My Drive/test.mp4" save=True

WARNING ⚠️ 'model' argument is missing. Using default 'model=yolov8n.pt'.
100% 6.23M/6.23M [00:00<00:00, 91.4MB/s]
requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 10.0s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

video 1/1 (frame 1/855) /content/drive/My Drive/test.mp4: 384x640 2 persons, 1 car, 1 motorcycle, 293.4ms
video 1/1 (frame 2/855) /content/drive/My Drive/test.mp4: 384x640 2 persons, 1 car, 1 motorcycle, 129.5ms
video 1/1 (frame 3/855) /content/drive/My Drive/test.mp4: 384x640 2 persons, 2 cars, 1 motorcycle, 127.3ms
video 1/1 (frame 4/855) /content/drive/My Drive/test.mp4: 384x640 2 person

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
from IPython.display import display, HTML

track_history = defaultdict(list)

model = YOLO("yolov8n.pt")
brake_model = YOLO("/content/drive/My Drive/best.pt")
names = model.model.names
brake_names = brake_model.model.names

video_path = "/content/drive/My Drive/test.mp4"
output_path = "/content/drive/My Drive/ggggg.avi"

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

# Process video frames
while cap.isOpened():
    success, frame = cap.read()
    if success:
        # Perform object tracking
        results = model.track(frame, persist=True, verbose=False)
        brake_results = brake_model(frame)

        boxes = results[0].boxes.xyxy.cpu()
        brake_boxes = brake_results[0].boxes.xyxy.cpu() if brake_results[0].boxes else []

        if results[0].boxes.id is not None:
            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            confs = results[0].boxes.conf.float().cpu().tolist()

            for box, cls, track_id in zip(boxes, clss, track_ids):
                # Update tracking history
                track = track_history[track_id]
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))
                if len(track) > 30:
                    track.pop(0)

                # Calculate speed and deviation
                if len(track) >= 2:
                    dx = track[-1][0] - track[-2][0]
                    dy = track[-1][1] - track[-2][1]
                    speed = np.sqrt(dx ** 2 + dy ** 2)
                    deviation = np.sqrt((dx / w) ** 2 + (dy / h) ** 2)
                else:
                    speed = 0
                    deviation = 0

                # Determine line color based on deviation
                line_color = (0, 0, 255) if deviation > 0.01 else (0, 255, 0)

                # Draw bounding box and text
                class_name = names[int(cls)] if 0 <= cls < len(names) else f"Class {cls}"
                cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
                cv2.putText(frame, f"Class: {class_name}, Speed: {speed:.2f}, Deviation: {deviation:.4f}",
                            (int(box[0]), int(box[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

                # Draw tracks
                for i in range(len(track) - 1):
                    cv2.line(frame, track[i], track[i + 1], line_color, 2)

            # Draw brake light detections
            for box in brake_boxes:
                cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 0, 255), 2)
                cv2.putText(frame, "Brake Light", (int(box[0]), int(box[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Write frame to video
        out.write(frame)

        # Display the frame in Colab
        display_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        display(display_frame)

        # Break on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# Release video capture and writer
cap.release()
out.release()
cv2.destroyAllWindows()



0: 256x416 1 Brake-Lights-OFF, 163.8ms
Speed: 2.1ms preprocess, 163.8ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 416)


array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   4],
        [  0,   0,   4],
        [  0,   0,   4]],

       [[  9,  11,  11],
        [  9,  11,  11],
        [  9,  11,  11],
        ...,
        [  0,   1,   6],
        [  0,   1,   6],
        [  0,   1,   6]],

       [[  9,  11,  11],
        [  9,  11,  11],
        [  9,  11,  11],
        ...,
        [  1,   0,  15],
        [  1,   1,  13],
        [  1,   1,  13]],

       ...,

       [[ 58,  69, 111],
        [ 58,  69, 111],
        [ 58,  69, 111],
        ...,
        [  4,   0,   9],
        [  4,   0,   9],
        [  4,   0,   9]],

       [[ 64,  70,  91],
        [ 64,  70,  91],
        [ 64,  70,  91],
        ...,
        [  3,   0,   6],
        [  3,   0,   6],
        [  3,   0,   6]],

       [[  0,   0,  20],
        [  0,   0,  20],
        [  0,   0,  20],
        ...,
        [  1,   0,   4],
        [  1,   0,   4],
        [  1,   0,   4]]


0: 256x416 1 Brake-Lights-OFF, 182.7ms
Speed: 2.2ms preprocess, 182.7ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 416)


array([[[  0,   1,   0],
        [  0,   1,   0],
        [  0,   1,   0],
        ...,
        [  0,   0,  10],
        [  0,   0,  10],
        [  0,   0,  10]],

       [[  0,   5,   4],
        [  0,   5,   4],
        [  0,   5,   4],
        ...,
        [  0,   0,  12],
        [  0,   0,  12],
        [  0,   0,  12]],

       [[  1,   6,   5],
        [  1,   6,   5],
        [  1,   6,   5],
        ...,
        [  1,   0,  19],
        [  1,   0,  19],
        [  1,   0,  19]],

       ...,

       [[ 58,  69, 111],
        [ 58,  69, 111],
        [ 58,  69, 111],
        ...,
        [  4,   0,   9],
        [  4,   0,   9],
        [  4,   0,   9]],

       [[ 64,  70,  91],
        [ 64,  70,  91],
        [ 64,  70,  91],
        ...,
        [  3,   0,   6],
        [  3,   0,   6],
        [  3,   0,   6]],

       [[  0,   0,  20],
        [  0,   0,  20],
        [  0,   0,  20],
        ...,
        [  1,   0,   4],
        [  1,   0,   4],
        [  1,   0,   4]]


0: 256x416 1 Brake-Lights-OFF, 168.0ms
Speed: 2.8ms preprocess, 168.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 416)


...

array([[[ 0,  0,  6],
        [ 0,  0,  6],
        [ 0,  0,  6],
        ...,
        [ 0,  1,  3],
        [ 0,  1,  3],
        [ 0,  1,  3]],

       [[50, 50, 58],
        [50, 50, 58],
        [50, 50, 58],
        ...,
        [ 8, 12, 14],
        [ 8, 12, 14],
        [ 8, 12, 14]],

       [[47, 52, 58],
        [47, 52, 58],
        [47, 52, 58],
        ...,
        [ 9, 13, 15],
        [ 9, 13, 15],
        [ 9, 13, 15]],

       ...,

       [[53, 58, 66],
        [50, 55, 63],
        [47, 52, 60],
        ...,
        [21, 23, 39],
        [21, 23, 39],
        [21, 23, 39]],

       [[50, 54, 58],
        [49, 53, 57],
        [46, 50, 54],
        ...,
        [23, 23, 33],
        [23, 23, 33],
        [23, 23, 33]],

       [[ 0,  4,  8],
        [ 0,  2,  6],
        [ 0,  0,  2],
        ...,
        [ 0,  0,  8],
        [ 0,  0,  8],
        [ 0,  0,  8]]], dtype=uint8)


0: 256x416 (no detections), 175.1ms
Speed: 2.1ms preprocess, 175.1ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 416)


array([[[ 0,  0,  6],
        [ 0,  0,  6],
        [ 0,  0,  6],
        ...,
        [ 0,  0,  4],
        [ 0,  0,  4],
        [ 0,  0,  4]],

       [[50, 50, 58],
        [50, 50, 58],
        [50, 50, 58],
        ...,
        [11, 12, 17],
        [11, 12, 17],
        [11, 12, 17]],

       [[47, 52, 58],
        [47, 52, 58],
        [47, 52, 58],
        ...,
        [14, 15, 20],
        [14, 15, 20],
        [14, 15, 20]],

       ...,

       [[56, 61, 71],
        [56, 61, 71],
        [56, 61, 71],
        ...,
        [ 8, 11, 20],
        [ 8, 11, 20],
        [ 8, 11, 20]],

       [[56, 60, 62],
        [56, 60, 62],
        [56, 60, 62],
        ...,
        [11, 12, 15],
        [11, 12, 15],
        [11, 12, 15]],

       [[ 0,  0,  2],
        [ 0,  0,  2],
        [ 0,  0,  2],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]]], dtype=uint8)


0: 256x416 (no detections), 185.0ms
Speed: 2.1ms preprocess, 185.0ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 416)


array([[[ 0,  0,  6],
        [ 0,  0,  6],
        [ 0,  0,  6],
        ...,
        [ 1,  2,  7],
        [ 0,  1,  6],
        [ 0,  0,  1]],

       [[50, 50, 58],
        [50, 50, 58],
        [50, 50, 58],
        ...,
        [32, 33, 38],
        [24, 25, 30],
        [18, 19, 24]],

       [[47, 52, 58],
        [47, 52, 58],
        [47, 52, 58],
        ...,
        [23, 24, 29],
        [17, 18, 23],
        [13, 14, 19]],

       ...,

       [[56, 61, 71],
        [56, 61, 71],
        [56, 61, 71],
        ...,
        [19, 27, 43],
        [19, 27, 43],
        [19, 27, 43]],

       [[56, 60, 62],
        [56, 60, 62],
        [56, 60, 62],
        ...,
        [21, 26, 34],
        [21, 26, 34],
        [21, 26, 34]],

       [[ 0,  0,  2],
        [ 0,  0,  2],
        [ 0,  0,  2],
        ...,
        [ 0,  0,  6],
        [ 0,  0,  6],
        [ 0,  0,  6]]], dtype=uint8)